In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
jdk_version = '11'
hadoop_version = 'hadoop2.7'

# Set Environment Variables
os.environ['SPARK_VERSION'] = spark_version
os.environ['JDK_VERSION'] = jdk_version
os.environ['HADOOP_VERSION'] = hadoop_version
os.environ['JAVA_HOME'] = f'/usr/lib/jvm/java-{jdk_version}-openjdk-amd64'
os.environ['SPARK_HOME'] = f'/content/{spark_version}-bin-{hadoop_version}'

# Update modules and install Spark and Java
!apt-get update
!apt-get install openjdk-$JDK_VERSION-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-$HADOOP_VERSION.tgz
!tar xf $SPARK_VERSION-bin-$HADOOP_VERSION.tgz
!pip install -q findspark

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://pp

In [2]:
postgresql_java_version = '42.2.9'
os.environ['POSTGRESQL_JAVA_VERSION'] = postgresql_java_version
!wget https://jdbc.postgresql.org/download/postgresql-$POSTGRESQL_JAVA_VERSION.jar

--2021-11-22 16:56:59--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.50MB/s    in 0.2s    

2021-11-22 16:56:59 (5.50 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Level-One-Starter-Code").config(
    "spark.driver.extraClassPath",
    f"/content/postgresql-{postgresql_java_version}.jar"
  ).getOrCreate()

# Load Amazon Data into Spark DataFrame

In [17]:
from pyspark import SparkFiles
raw_data_file = "amazon_reviews_us_Video_Games_v1_00.tsv.gz"
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/" + raw_data_file
spark.sparkContext.addFile(url)
# Read and show the data
df = spark.read.option("sep", "\t").csv("file://" + SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), header=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [10]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [11]:
# What is the size of Data?
print((df.count(), len(df.columns)))

(1785997, 15)


# Cleaned up DataFrames to match tables

In [18]:
from pyspark.sql.functions import to_date
# Review DataFrame
df = df.withColumn('review_date',to_date(df.review_date, 'yyyy-MM-dd'))
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [19]:
from pyspark.sql.types import IntegerType
df = df.withColumn("star_rating", df["star_rating"].cast(IntegerType()))
df = df.withColumn("helpful_votes", df["helpful_votes"].cast(IntegerType()))
df = df.withColumn("total_votes", df["total_votes"].cast(IntegerType()))
df = df.withColumn("customer_id", df["customer_id"].cast(IntegerType()))
df = df.withColumn("product_parent", df["product_parent"].cast(IntegerType()))
df = df.withColumn("customer_id", df["customer_id"].cast(IntegerType()))

df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



# Push to AWS RDS instance

In [22]:
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [27]:
mode = "append"
jdbc_url = "jdbc:postgresql://bigdatadb.clfyppfq4ufq.us-east-2.rds.amazonaws.com:5432/bigData_db"
config = {
    "user" : "user", 
    "password" : "password",
    "driver" : "org.postgresql.Driver"
}

In [28]:
# Write review_id_df to table in RDS
review_id_df = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df = df.orderBy('product_id').select(['product_id', 'product_title'])
products_df = products_df.dropDuplicates()
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [49]:
# Write customers_df to table in RDS
customers_df = df.groupBy('customer_id').count()
customers_df = customers_df.withColumnRenamed('count', 'customer_count')
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [51]:
# Write vine_df to table in RDS
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
# vine_df.show(5)
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)